Thanks to all the contributors with kernels to this competition!

Specially, to the ones that provided code and comments that inspired us making this and other kernels we developed in the context of this competition: 

https://www.kaggle.com/dmdm02/complete-eda-voting-lightgbm 

https://www.kaggle.com/chocozzz/two-sigma-news-simple-eda-prophet-nlp 

https://www.kaggle.com/ashishpatel26/bird-eye-view-of-two-sigma-nn-approach 
    
https://www.kaggle.com/jsaguiar/baseline-with-news 
    
https://www.kaggle.com/artgor/eda-feature-engineering-and-everything 
    
https://www.kaggle.com/christofhenkel/market-data-nn-baseline 
    
https://www.kaggle.com/smasar/tutorial-timeseriesapproach 
    
https://www.kaggle.com/rabaman/0-64-in-100-lines 
    
https://www.kaggle.com/guowenrui/market-nn-if-you-like-you-can-use-it-and-upvote/notebook 
    
https://www.kaggle.com/smasar/eda-preprocessing-processing-evaluation

Importar librerías

In [ ]:
import pandas as pd
import numpy as np
import time

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import collections

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

import datetime

plt.style.use('seaborn')
sns.set(font_scale=2)

import warnings 
warnings.filterwarnings('ignore')
import os

Crear entorno

In [ ]:
from kaggle.competitions import twosigmanews
# You can only call make_env() once, so don't lose it!
env = twosigmanews.make_env()
print('Done!')

Cargar datos de training

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()

In [ ]:
print("In total: ", news_train_df.shape)

In [ ]:
#news_train_df.head()
news_train_df.tail()

Cargar datos de test

In [ ]:
days = env.get_prediction_days()
(market_test_df, news_test_df, predictions_template_df) = next(days)

In [ ]:
print("In total: ", news_test_df.shape)

In [ ]:
news_test_df.head()

Visualizar plantilla de predicción

In [ ]:
predictions_template_df.head()

In [ ]:

print("In news_train_df: ", news_train_df.shape);print("In news_test_df: ", news_test_df.shape)

Dadas las características del proyecto, de cara a poder medir la accuracy, simplemente consultamos las características de los ficheros de test... pero no vamos a usarlos en lo sucesivo, sino que vamos a coger un subconjunto de los ficheros de train, que tengan las mismas características: los datos de mercados del último día y los datos del fichero de noticias posteriores a las 22h del penúltimo día (más todos los del último día). 

Nota: Esto creo que no lo hacemos hasta que vayamos a modelar. El exploratorio no hay por qué no hacerlo sobre el dataset completo. PTE hacer lo mismo en con los datos de Market y MERGE

In [ ]:
#del market_test_df, news_test_df

#start = '2016-12-29 22:00:00+00:00'

#market_test_df = market_train_df.loc[market_train_df['time'] > start]
#news_test_df = news_train_df.loc[news_train_df['time'] > start]

#market_train_df = market_train_df.loc[market_train_df['time'] <= start].reset_index(drop=True)
#news_train_df = news_train_df.loc[news_train_df['time'] <= start].reset_index(drop=True)

# Análisis Univariante Fichero de Noticias

A continuación, analizaremos variable por variable el fichero de noticias

* **TIME**


time(datetime64[ns, UTC]) - marca de tiempo UTC ("Universal Time Coordinated", hora del meridiano Greenwich) que muestra cuándo los datos estuvieron disponibles en la fuente

In [ ]:
news_train_df.time.describe()

In [ ]:
fig,axes = plt.subplots(1,1,figsize=(20,10))
axes.set_title("Time Distribution")
axes.set_ylabel("# of records")
axes.set_xlabel("date")
axes.plot(news_train_df.time.dt.date.value_counts().sort_index().index, news_train_df.time.dt.date.value_counts().sort_index().values)

Lo más destacable son los mínimos que hay cada año, coincidentes con el periodo de navidad. También es destacable el incremento de noticias que hay regularmente coincidiendo con cada fin de trimestre. Sería importante ver si este incremento se debe a un aumento de notas informativas periódicas de algún tipo determinado, ya que no es lo mismo una nota de resultados financieros de fin de trimestre, que una nota descriptiva de lo que ha pasado en el mercado (que por definición ya no tendrían ya valor predictivo)


* **SOURCE_TIMESTAMP**
sourceTimestamp represents the time when the news was created. Lets do a studying like a previous feature on this attribute.

In [ ]:
news_train_df.sourceTimestamp.describe()

In [ ]:
fig,axes = plt.subplots(1,1,figsize=(20,10))
axes.set_title("sourceTimestamp Distro")
axes.set_ylabel("# of records")
axes.set_xlabel("date")
axes.plot(news_train_df.sourceTimestamp.dt.date.value_counts().sort_index().index, news_train_df.sourceTimestamp.dt.date.value_counts().sort_index().values)

Se ve una distribución muy similar a la variable time,  quizás una de las dos variables sea prescindible, a falta de profundizar en analizar la diferencia entre ambas variables y analizarla.

* **FIRST_CREATED**

firstCreated(datetime64[ns, UTC]) - marca de tiempo UTC de esta noticia cuando se creó.

In [ ]:
news_train_df.firstCreated.describe()

In [ ]:
fig,axes = plt.subplots(1,1,figsize=(20,10))
axes.set_title("firstCreated Distro")
axes.set_ylabel("# of records")
axes.set_xlabel("date")
axes.plot(news_train_df.firstCreated.dt.date.value_counts().sort_index().index, news_train_df.firstCreated.dt.date.value_counts().sort_index().values)

Aplica el mismo comentario anterior

 **SOURCE_ID**

Es una variable que asigna una identificación para cada noticia

In [ ]:
news_train_df.sourceId.value_counts().head(10)

In [ ]:
news_train_df.sourceId.describe()

In [ ]:
news_train_df.sourceId.value_counts().describe()

Hay 6.340.206  artículos únicos. El 75% of no se repiten, y el resto se repite un número pequeño de repeticiones. El máximo que se repite alguno es 43 times. Los que se repiten son aquellos en los que está involucrado más de un asset name, pero habría que verificarlo en el análisis relacional

* **HEADLINE**
El titular de la noticia

In [ ]:
news_train_df.headline.describe()

Como en el número de artículos, hay repeticiones. Hay 5.532.379 titulares únicos.

Podemos ver los titulares que más se repiten:

In [ ]:
news_train_df['headline'].head(20)

Dividimos los datos en dos conjuntos, uno para las alertas y otro para los artículos

In [ ]:
news_train_df['headline'].value_counts()

alerts_df = news_train_df[news_train_df.urgency == 1]
article_df = news_train_df[news_train_df.urgency == 3]

Vemos las palabras más frecuentes de las alertas:

In [ ]:
text = ' '.join(alerts_df.headline.values[:1000])
stop  = list(STOPWORDS)
wordcloud = WordCloud(max_font_size=None, stopwords=stop, background_color='black',
                      width=1500, height=1200).generate(text)
plt.figure(figsize=(12, 8))
plt.imshow(wordcloud)
plt.title("Words frequented in Alerts Headlines", fontsize=20)
plt.imshow(wordcloud.recolor(colormap= 'viridis' , random_state=17), alpha=0.98)
plt.show()

Vemos las palabras más frecuentes de los artículos

In [ ]:
text2 = ' '.join(article_df.headline.values[:700])
stop  = list(STOPWORDS)
wordcloud = WordCloud(max_font_size=None, stopwords=stop, background_color='black',
                      width=1200, height=1000).generate(text2)
plt.figure(figsize=(12, 8))
plt.imshow(wordcloud)
plt.title("Words frequented in Article Headlines", fontsize=20)
plt.imshow(wordcloud.recolor(colormap= 'inferno' , random_state=17), alpha=0.98)
plt.show()

Esta variable es un texto, que no usaremos en los modelos, puesto que el dataset ya contiene extráido el análisis de Text Mining que nos aporta variables numéricas y de sentimiento útiles para el modelo

* **URGENCY**

Diferencia tipos de historia en función de la urgencia/importancia (1: alerta, 3: artículo)

In [ ]:
news_train_df.urgency.value_counts()

In [ ]:
(news_train_df['urgency'].value_counts() / 1000000).plot('bar');
plt.xticks(rotation=30);
plt.title('Urgency counts (mln)');

Vemos que hay unos dos tercios de artículos y el tercio restante de alertas.

En principio, intuitivamente pensamos que las alertas deben influir más en los cambios de cotizaciones, porque muchos de los artículos pueden ser descriptivos de la realidad de los mercados, incluyendo las subidas o bajadas ya efectivamente ocurridas.

Los 22 unknown, parecen error en los datos. Limpiaremos el dataset para el modelado.

* **TAKE_SEQUENCE**

El número de secuencia de la noticia sobre un mismo activo, comenzando en 1, para cuando una historia está en varios items porque van ocurriendo a lo largo del tiempo y en ese tiempo la agencia va haciendo actualizaciones. Para una historia determinada, las alertas y los artículos tienen secuencias separadas.

In [ ]:
news_train_df.takeSequence.value_counts().head(20)

Se comprueba que la inmensa mayoría de noticias no pasan de la primera secuencia. Tan sólo 7.099 llegan a la secuencia 20. No parece necesaria acción sobre esta variable, posiblemente sea irrelevante

* **PROVIDER**

Este campo es un identificador de la organización que proporcionó la noticia (por ejemplo, RTRS para Reuters News, BSW for Business Wire) 

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(15,10))
ax.set_xlabel("name")
ax.set_ylabel("#")
news_train_df.provider.value_counts().plot(kind="bar",legend="provider",color="tan")

Vemos como la inmensa mayoría de las noticias provienen de Reuters. Este campo no parece que vaya a tener ninguna relevancia, por su contenido y distribución de valores

*  **HeadlineTag(object)** 

the Thomson Reuters headline tag for the news item

In [ ]:
news_train_df['headlineTag'].value_counts()

* **SUBJECTS**

Códigos de temas e identificadores de empresas relacionados con esta noticia. Los códigos de tema describen el tema de la noticia. Estos pueden cubrir clases de activos, geografías, eventos, industrias / sectores y otros tipos

In [ ]:
news_train_df.subjects.head(10)

In [ ]:
news_train_df.subjects.value_counts().head(50)

In [ ]:
from collections import Counter
tmp_list = []
for i in news_train_df.head(1000).subjects:
    tmp_list += i.replace("{","").replace("}","").replace(" ","").split(",")

Counter(tmp_list)
fig,ax = plt.subplots(1,1,figsize=(30,10))
ax.set_xticklabels(dict(Counter(tmp_list)).keys(),rotation=70)
font = {'family' : 'normal',
         'weight' : 'bold',
         'size'   : 2}
plt.rc('font', **font)
ax.bar(dict(Counter(tmp_list)).keys(),dict(Counter(tmp_list)).values())

Esta variable difícilmente puede tener influencia en modelos de ML tal y como está informada. Es una variable categórica en donde hay una amplia variedad de posibles valores, que a su vez son listas de todas las temáticas que incluyen que parecen genéricas. 

Los valores que presentan las listas son genéricas ("Energía", "Europa", "Reuters", "Company"...etc). El tratamiento para poder explotar esta variable potencialmente es complejo (ej. separar los distintos temas de cada noticia, hacer un diccionario total de subjects y resumir en cada noticia lo más relevante). Parece que saldrían muchos tipos para manipular los datos y no le vemos mucha probabilidad de que tenga relevancia por lo genéricas que son.

* **AUDIENCES**

Identifica a qué tipo o categoría genera de audiencia pertenece la noticia. Por lo general, se adaptan a audiencias específicas. (por ejemplo, "M" para Money International News Service y "FB" para French General News Service)

In [ ]:
news_train_df.head(10).audiences

In [ ]:
from collections import Counter
tmp_list = []
for i in news_train_df.head(100000).audiences:
    tmp_list += i.replace("{","").replace("}","").replace(" ","").split(",")

Counter(tmp_list)

fig,ax = plt.subplots(1,1,figsize=(30,10))
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 10}
plt.rc('font', **font)
ax.set_xticklabels(dict(Counter(tmp_list)).keys(),rotation=60)
ax.bar(dict(Counter(tmp_list)).keys(),dict(Counter(tmp_list)).values())

In [ ]:
news_train_df['audiences'].value_counts().head(50)

Igual que el campo anterior, el tratamiento potencial de este campo para que pueda tener valor es complejo. Dado lo genérico del contenido, y que intuitivamente, el significado del campo,no parece que sea relevante (lo importante debería ser la noticia, no la audiencia a quien va destinada), no lo manipulamos. 

* ** Variables Numéricas: BODY_SIZE, COMPANY_COUNT,SENTENCE_COUNT,WORD_COUNT**

Analizamos variables numéricas

bodySize - > El tamaño de la versión actual del cuerpo de la noticia (tamaño en número de caracteres).

companyCount -> el número de empresas enumeradas explícitamente en la noticia en el campo asunto

sentenceCount -> el número total de frases en la noticia. Puede usarse junto con firstMentionSentence para determinar la posición relativa de la primera mención en el elemento.

wordCount -> El número total de tokens léxicos (palabras y puntuación) en la noticia

In [ ]:
pd.concat([news_train_df.bodySize.describe(),news_train_df.companyCount.describe(),news_train_df.sentenceCount.describe(),news_train_df.wordCount.describe()],axis=1)

Las noticias tienen de media unos 3.800 caracteres, y hay algún registro sin caracteres (a eliminar)

De media una noticia nombra 5 compañías, y llegan mencionarse hasta 9. 

In [ ]:
plt.violinplot(news_train_df['bodySize'])
plt.ylim(-0.5, 10000)
plt.yticks(size = 15)
plt.xticks(size = 15)
plt.show()

In [ ]:
plt.violinplot(news_train_df['companyCount'])
plt.ylim(0, 10)
plt.yticks(size = 15)
plt.xticks(size = 15)
plt.show()

Es posible que el hecho de nombrar más de una empresa por noticia limite las capacidades predicitivas, al correlacionarlo con los datos de mercado, se puede "diluir" el comentario entre todas las empresas que menciona el artículo. Creemos que puede limitar mucho el uso del cuerpo de las noticias.

* **HEADLINE_TAG**

El titular de la etiqueta para la noticia para Thomson Reuters 

In [ ]:
news_train_df['headlineTag'].value_counts().head(20)

Para hacernos una idea general del tipo de información que aporta este campo, usamos un gráfico de wordcount

In [ ]:
text=""
text =" ".join(list(news_train_df.headlineTag))
wordcloud = WordCloud(width=1024, height=1024, margin=0).generate(text)
 
# Display the generated image:
fig,ax = plt.subplots(1,1,figsize=(10,10))
ax.imshow(wordcloud, interpolation='bilinear')
ax.axis("off")
ax.margins(x=0, y=0)
plt.show()

Parece que las noticias tienen en sus titulares muy frecuentemente palabras muy genéricas

In [ ]:
(news_train_df['headlineTag'].value_counts() / 1000)[:30].plot('barh');
plt.xlim(0, 1000)
plt.yticks(size = 6)
plt.xticks(size = 6)
plt.title('headlineTag counts (thousands)',size=12)

La mayoría de los campos no tienen etiqueta (ya lo habíamos visto en el exploratorio general). 

La mayoría de las noticias que tienen etiqueta son parecen recurrentes (ej, "update", "buzz", "market pulse", "table") y poco vinculados a novedades de última hora que podrían tener más impacto, como por ejemplo "exclusive".  En teoría deberían tener más influencia noticias que supongan novedades y análisis 

* **MARKET_COMMENTARY**

Esta variable es un indicador de que el artículo está discutiendo las condiciones generales del mercado diarias (como por ejemplo los resúmenes "After the bell)"

In [ ]:
news_train_df.marketCommentary.value_counts()

In [ ]:
market_commentary_df_dict = dict(collections.Counter(list(news_train_df.marketCommentary)))
percent_commentary_df_dict = {k: v / total for total in (sum(market_commentary_df_dict.values()),) for k, v in market_commentary_df_dict.items()}
explode=(0,0.1)
labels ='False','True'
fig, ax = plt.subplots(1,1, figsize=(8,8))
ax.set_title("Representing Genral Market Conditions")
ax.pie(list(percent_commentary_df_dict.values()), explode=explode, labels=labels, autopct='%1.1f%%',shadow=True, startangle=90)

Solo el 5% de las noticias son de esta categoría, aunque hay algunos valores de Headline Tag, que hemos visto antes, que parecen comentarios generales, por lo que este indicador no parece muy fiable ni relevante

* **ASSET_CODE**

Lista de activos mencionados en la noticia

In [ ]:
news_train_df.assetCodes.head(10)

Vemos cómo en cada registro de noticias hay una lista de asset codes.

Como esta es la clave del registro de mercados, habrá que hacer algún tipo de tratamiento para poder unir ambos ficheros.

Esto lo haremos en el apartado de preparación de los datos, previo a la elaboración de los modelos.

 **ASSET_NAME**

Nombre del activo

In [ ]:
news_train_df['assetName'].head(10)

In [ ]:
news_train_df['assetName'].value_counts()

Nos hacemos de nuevo una idea general de los valores, con el gráfico WordCount

este campo no añade nada sustancial al asset code. Algunos asset names comprenden varios asset codes y podría alternativamente pensarse en un modelo por assetname argumentando que las noticias afectan a todos los valores cotizados del grupo empresarial, pero complicaría el modelo y no lo consideramos momentáneamente 

In [ ]:
tmp_list = []
for i in news_train_df.head(200000).assetName:
    tmp_list += i.replace("{","").replace("}","").replace(" ","").split(",")
text =" ".join(tmp_list).replace("'","")
 # Create the wordcloud object
wordcloud = WordCloud(width=1024, height=1024, margin=0).generate(text)
 
# Display the generated image:
fig,ax = plt.subplots(1,1,figsize=(10,10))
ax.imshow(wordcloud, interpolation='bilinear')
ax.axis("off")
ax.margins(x=0, y=0)
plt.show()

* **FIRST_MENTION_SENTENCE**

La primera frase, que comienza con el titular, en el que se menciona el activo calificado

*	1: titular
*	2: primera frase del cuerpo de la noticia
*	3: segunda frase del cuerpo de la noticia
*	0: el activo que se calificó no se encontró en el encabezado de la noticia o en el texto del cuerpo.

Como resultado, el texto completo de la noticia (título + cuerpo) se utilizará para determinar el sentimiento


In [ ]:
news_train_df.firstMentionSentence.describe()

In [ ]:
plt.violinplot(news_train_df['firstMentionSentence'])
plt.ylim(0, 30)
plt.yticks(size = 15)
plt.xticks(size = 15)
plt.show()

Esta distribución nos indica que la probabilidad de que salga mencionado el activo en las primeras líneas es muy elevada y decae uniformemente

* **RELEVANCE**

Un número decimal que indica la relevancia de la noticia para el activo. El rango va de 0 a 1. Si el activo se menciona en el titular, la relevancia se establece en 1. Cuando el elemento es una alerta (urgencia == 1), la relevancia debe medirse mejor por la variable firstMentionSentence.

In [ ]:
news_train_df.relevance.describe()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(9, 9))
axes.set_title("Volume Violin")
axes.set_ylabel("Volume")
axes.violinplot(list(article_df["relevance"].values),showmeans=False,showmedians=True)

Se puede ver, tanto con el diagrama de Violin como en con el describe, cómo predominan los 1 en los artículos (hemos dejado fuera las alertas)

¿Podríamos quizás intentar detectar alguna relación entre esta variable y la de wordcount y sentimentwordcount?

* **SENTIMENT_CLASS**

Indica la clase de sentimiento predominante para esta noticia con respecto al activo. La clase indicada es la de mayor probabilidad

In [ ]:
news_train_df.sentimentClass.value_counts()

Los valores de esta variable son:

* sentimentClass = 1 : Sentimiento positivo
* sentimentClass = -1 : Sentimiento negativo
* sentimentClass = 0 : Sentimiento neutro

In [ ]:
sentiment_df_dict = dict(collections.Counter(list(news_train_df.sentimentClass)))
percent_univers_df_dict = {k: v / total for total in (sum(sentiment_df_dict.values()),) for k, v in sentiment_df_dict.items()}
explode=(0.0,0.025,0.05)
labels ='1','0',"-1"
fig, ax = plt.subplots(1,1, figsize=(8,8))
ax.set_title("sentimentClass")
ax.pie(list(percent_univers_df_dict.values()), explode=explode, labels=labels, autopct='%1.1f%%',shadow=True, startangle=90)

La distribución del sentimentClass es bastante uniforme, aunque destaca ligeramente el sentimiento negativo

Sería interesante llevarse esta variable junto con las que daban un valor para cada tipo de sentimiento, para ver cómo se relacionan:

* **Análisis de variables de Sentimiento: Sentiment Class & SENTIMENT_NEGATIVE, SENTIMENT_NEUTRAL, SENTIMENT_POSITIVE**

sentimentNegative -> Probabilidad de que el sentimiento de la noticia sea negativo para el activo.

sentimentNeutral -> Probabilidad de que el sentimiento de la noticia sea neutral para el activo.

sentimentPositive -> Probabilidad de que el sentimiento de la noticia sea positivo para el activo.


In [ ]:
pd.concat([news_train_df.assetName.head(10),news_train_df.sentimentClass.head(10),news_train_df.sentimentPositive.head(10),news_train_df.sentimentNeutral.head(10),news_train_df.sentimentNegative.head(10)],axis = 1)

In [ ]:
pd.concat([news_train_df.sentimentNegative.describe(),news_train_df.sentimentNeutral.describe(),news_train_df.sentimentPositive.describe()],axis=1)

In [ ]:
fig , axes = plt.subplots(1,1,figsize=(20,8))
news_train_df.sentimentNegative.head(100).plot(kind="bar",legend="Negative",colormap="brg")
news_train_df.sentimentPositive.head(100).plot(colormap="Set2",linewidth=2,legend="Postive")
news_train_df.sentimentNeutral.head(100).plot(colormap="RdGy",linewidth=0.8,linestyle='dashed',legend="Neutral")
plt.yticks(size = 8)
plt.xticks(size = 8)
font = {'family' : 'normal',
        'weight' : 'bold',
        'size'   : 10}
axes.set_xticklabels(news_train_df.head(100).index,rotation=90)
legend = axes.legend(loc='upper left', shadow=True, fontsize='small')
plt.rc('font', **font)

La comparación entre las probabilidades de sentimientos negativos, positivos y neutrales consiste en identificar el predominante. 

No se ve una relación entre estas probabilidades. Pueden haber noticias de distinto signo conviviendo en todo momento. Por ejemplo en los registros, 83, 84, y 85 los efectos negativo y positivo son bajos pero el neutral es alto. En contraste los registros 53, y 54, el positivo es el mayor.

El sentimentClass nos indica el sentimiento predominante. A priori podríamos pensar que tiene más relevancia sentimientos positivos o negativos que los neutros, ya que los inversores están más atentos a novedades para tomar sus decisiones de inversión, y parece lógico pensar que es más probable que los retornos (variable objetivo) varíen ante eventos imprevistos o prospecciones de negocio negativos o positivos, aunque el resto de noticias sean neutras. 

¿Consideramos dar más peso a los sentimientos negativos o positivo para intentar ver la relación con la variable objetivo a través del sentiment class o una variable simliar?



* **SENTIMEN_WORD_COUNT**

El número de tokens en las secciones del texto del elemento que se consideran relevantes para el activo. Esto se puede usar junto con wordCount para determinar la proporción de la noticia que discute el activo.

In [ ]:
pd.concat([news_train_df.assetName.head(10),news_train_df.sentimentClass.head(10),news_train_df.sentimentWordCount.head(10)],axis = 1)


In [ ]:
news_train_df.sentimentWordCount.describe()

In [ ]:
plt.violinplot(news_train_df['sentimentWordCount'])
plt.ylim(0, 1000)
plt.yticks(size = 15)
plt.xticks(size = 15)
plt.show()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8))
ax.set_title("Hist(log sentimentWordCount)")
ax.set_xlabel("Log(sentimentWordCount)")
np.log10(news_train_df.sentimentWordCount).hist(ax=ax,)

Vemos tanto en el describe como en el diagrama de violin e histograma, que en media hay menos 200 tokens relacionados con el activo y una distribución tendente a ser uniforme

Creemos que esta variable puede ser relevante si queremos determinar impacto de artículos en los rendimientos, ya que nos ayudarían a relativizar (noticias que relativamente indican un sentimiento más veces que el resto de activos mencionados)


* **Indicador de noticia novedosa: NOVELTY_COUNT12H, NOVELTY_COUNT_24H, NOVELTY_COUNT_3D, NOVELTY_COUNT_5D, NOVELTY_COUNT_7D**


La novedad en un intervalo de X horas del contenido dentro de alguna noticia para el un activo a que se refiere la fila****. 

Se calcula comparándolo con el texto específico del activo en un caché de elementos de noticias anteriores que contienen el activo

In [ ]:
pd.concat([news_train_df.assetName.head(10),news_train_df.sentimentClass.head(10), news_train_df.noveltyCount12H.head(10),news_train_df.noveltyCount24H.head(10),news_train_df.noveltyCount3D.head(10),news_train_df.noveltyCount5D.head(10),news_train_df.noveltyCount7D.head(10)],axis = 1)

In [ ]:
pd.concat([news_train_df.noveltyCount12H.describe(),news_train_df.noveltyCount24H.describe(), news_train_df.noveltyCount3D.describe(),
          news_train_df.noveltyCount5D.describe(),news_train_df.noveltyCount7D.describe()],axis=1)

In [ ]:
fig,axes = plt.subplots(3,2,figsize=(10,15))

#A counter is a container that stores elements as dictionary keys, and their counts are stored as dictionary values.

noveltyCount12H_dict = dict(collections.Counter(list(news_train_df.noveltyCount12H)))
percent_noveltyCount12H_dict = {k: v / total for total in (sum(noveltyCount12H_dict.values()),) for k, v in noveltyCount12H_dict.items()}
sizes = list(percent_noveltyCount12H_dict.values())

axes[0][0].set_title("noveltyCount12H",loc="left")
axes[0][0].pie(sizes,  autopct='%1.1f%%',shadow=False, startangle=90)


noveltyCount24H_dict = dict(collections.Counter(list(news_train_df.noveltyCount24H)))
percent_noveltyCount24H_dict = {k: v / total for total in (sum(noveltyCount24H_dict.values()),) for k, v in noveltyCount24H_dict.items()}
sizes = list(percent_noveltyCount24H_dict.values())
axes[0][1].set_title("noveltyCount24H",loc="left")
axes[0][1].pie(sizes,  autopct='%1.1f%%',shadow=False, startangle=90)

noveltyCount3D_dict = dict(collections.Counter(list(news_train_df.noveltyCount3D)))
percent_noveltyCount3D_dict = {k: v / total for total in (sum(noveltyCount3D_dict.values()),) for k, v in noveltyCount3D_dict.items()}
sizes = list(percent_noveltyCount3D_dict.values())
axes[1][0].set_title("noveltyCount3D",loc="left")
axes[1][0].pie(sizes,  autopct='%1.1f%%',shadow=False, startangle=90)


noveltyCount5D_dict = dict(collections.Counter(list(news_train_df.noveltyCount5D)))
percent_noveltyCount5D_dict = {k: v / total for total in (sum(noveltyCount5D_dict.values()),) for k, v in noveltyCount5D_dict.items()}
sizes = list(percent_noveltyCount5D_dict.values())
axes[1][1].set_title("noveltyCount5D",loc="left")
axes[1][1].pie(sizes,  autopct='%1.1f%%',shadow=False, startangle=90)

noveltyCount7D_dict = dict(collections.Counter(list(news_train_df.noveltyCount7D)))
percent_noveltyCount7D_dict = {k: v / total for total in (sum(noveltyCount7D_dict.values()),) for k, v in noveltyCount7D_dict.items()}
sizes = list(percent_noveltyCount7D_dict.values())
axes[2][0].set_title("noveltyCount7D",loc="left")
axes[2][0].pie(sizes,  autopct='%1.1f%%',shadow=False, startangle=90)


overal_dict = pd.concat([news_train_df.noveltyCount12H,news_train_df.noveltyCount24H, news_train_df.noveltyCount3D,
          news_train_df.noveltyCount5D,news_train_df.noveltyCount7D],axis=0)
noveltyOveral_dict = dict(collections.Counter(list(overal_dict)))
percent_overal_dict = {k: v / total for total in (sum(noveltyOveral_dict.values()),) for k, v in noveltyOveral_dict.items()}
sizes = list(percent_overal_dict.values())
axes[2][1].set_title("overalNovelty",loc="left")
axes[2][1].pie(sizes,  autopct='%1.1f%%',shadow=False, startangle=90)
print()

In [ ]:
print(noveltyCount12H_dict)

La mayoría de noticias no tienen menciones previas (53%) , y la proporción va decreciendo.

Vemos que se mantienen estables a lo largo del tiempo las proporciones de activos que salen mencionadas 1 o sucesivas veces. Por ejemplo, las que son mencionadas una vez en las doce horas anteriores, son el 17,5%, y las que son mencionadas una vez en los 7 días anteriores , son 17,8%.


* **Cantidad de noticias que mencionan el activo: VOLUME_COUNT_12H, VOLUME_COUNT_24H, VOLUME_COUNT_3D, VOLUME_COUNT_15D, VOLUME_COUNT_7D**

El volumen de 12 horas de noticias para cada activo. Se mantiene un caché de las noticias anteriores y se calcula la cantidad de noticias que mencionan el activo.

In [ ]:
pd.concat([news_train_df.assetName.head(10),news_train_df.sentimentClass.head(10), news_train_df.volumeCounts12H.head(10),news_train_df.volumeCounts24H.head(10),news_train_df.volumeCounts3D.head(10),
        news_train_df.volumeCounts5D.head(10), news_train_df.volumeCounts7D.head(10)],axis = 1)


In [ ]:
pd.concat([news_train_df.volumeCounts12H.describe(),news_train_df.volumeCounts24H.describe(), news_train_df.volumeCounts3D.describe(),
          news_train_df.volumeCounts5D.describe(),news_train_df.volumeCounts7D.describe()],axis=1)

In [ ]:

plt.boxplot(list(news_train_df["volumeCounts12H"].values))
plt.ylim(0, 25)
plt.yticks(size = 15)
plt.xticks(size = 15)
plt.show()

plt.boxplot(list(news_train_df["volumeCounts24H"].values))
plt.ylim(0, 40)
plt.yticks(size = 15)
plt.xticks(size = 15)
plt.show()

plt.boxplot(list(news_train_df["volumeCounts3D"].values))
plt.ylim(0, 60)
plt.yticks(size = 15)
plt.xticks(size = 15)
plt.show()

plt.boxplot(list(news_train_df["volumeCounts5D"].values))
plt.ylim(0, 80)
plt.yticks(size = 15)
plt.xticks(size = 15)
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=2, figsize=(10, 15),squeeze=False)
axes[0][0].set_title("volumeCounts12H")
axes[0][0].violinplot(list(news_train_df["volumeCounts12H"].values))

axes[0][1].set_title("volumeCounts24H")
axes[0][1].violinplot(list(news_train_df["volumeCounts24H"].values))

axes[1][0].set_title("volumeCounts3D")
axes[1][0].violinplot(list(news_train_df["volumeCounts3D"].values))

axes[1][1].set_title("volumeCounts5D")
axes[1][1].violinplot(list(news_train_df["volumeCounts5D"].values))

axes[2][0].set_title("volumeCounts7D")
axes[2][0].violinplot(list(news_train_df["volumeCounts7D"].values))

fig.delaxes(axes[2][1])

Está claro que al aumentar la marca de tiempo (que va de 12H a 7D) se incrementa el rango de VolumeCounts. 

Creemos que tienen más valor aquellos registros con mayor volumencounts en los primeros días, aunque también podría haber un efecto sobre las cotizaciones aquellas secuencias que mantienen al activo en las noticias con mucha presencia en relación a la media

En el análisis relacional bivariante podemos ver la relación entre valores altos de estas variables y los sentimientos, y entre estas variables y las empresas. Intuitivamente pensamos que si se habla muchos días de un activo ha de estar sesgado hacia sentimientos negativos o positivos , aunque no hay que descartar que simplemente haya valores que estén más presentes en los medios continuamente, sin grandes sesgos hacia mensajes positivos o negativos.

A priori creemos que a medida que pasa el tiempo y los activos son mencionados, incluso muchas veces, es poco probable que influyan en los resultados, a excepción quizás de situaciones excepcionales de crisis o empresas en situación de vigilancia especial (fondos con altas posiciones a corto que esperan noticias para tomar decisiones